In [3]:
import os
import re
import csv
import operator
import collections
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix

In [4]:
# split word to ( , )

train_dir = 'Mail_Plain_Content'
emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]

all_words = []
for email in emails:
    with open(email, encoding = 'ISO-8859-1') as m:
        a = []
        for i,line in enumerate(m):
            if i != 1:
                words_ = line
                pattern3 = re.compile("[^\w\d]+")
                words_ = pattern3.sub(' ',words_)
                words = words_.split()
                for j in range(len(words)):
                    a.append(words[j])
    all_words.append(a)
print ('size of letter :', len(all_words))

size of letter : 8915


In [5]:
import numpy as np

np.save('all_words.npy', all_words)

In [6]:
cut_programs = np.load('all_words.npy')

### generate dictionary

In [12]:
# generate dictionary

word_dict = dict()

In [13]:
def add_word_dict(w):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

In [14]:
for letter in cut_programs:
    for w in letter:
        add_word_dict(w)

In [15]:
import operator

word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
print ('size of word :', len(word_dict))

size of word : 106281


In [33]:
# choose word by frequence 10 < f

VOC_SIZE = 15000
VOC_START = 650

voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
print(voc_dict[:10])
np.save('voc_dict.npy', voc_dict)

[('ID', 526), ('code', 526), ('Aug', 525), ('²æ', 524), ('support', 523), ('United', 522), ('Free', 522), ('offers', 520), ('loss', 518), ('next', 515)]


In [34]:
voc_dict = np.load('voc_dict.npy')

In [35]:
# word filter

example_doc = []

for letter in cut_programs:
    example_line = ''
    for w in letter:
        if w in voc_dict:
            example_line += w+' '
        
    example_doc.append(example_line)

print( example_doc[0] )

Chris working outside UK Fire Industry lifestyle decision Chris part leaves wishes Senior Management Team resend sales One Sales Team within 24 hours require immediate assistance Office Manager Paul Technical Support able fully assist Many thanks humble temporary inconvenience Yours sincerely Chris UK Sales Distribution Manager 


### word2vec

In [36]:
# import modules & set up logging
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [37]:
# train word2vec on the two sentences
model = gensim.models.Word2Vec(example_doc, min_count=1, size=200, hs=1, negative=0)

2017-10-28 20:03:38,315 : INFO : collecting all words and their counts
2017-10-28 20:03:38,317 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2017-10-28 20:03:38,318 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-28 20:03:38,670 : INFO : collected 133 word types from a corpus of 6014253 raw words and 8915 sentences
2017-10-28 20:03:38,671 : INFO : Loading a fresh vocabulary
2017-10-28 20:03:38,671 : INFO : min_count=1 retains 133 unique words (100% of original 133, drops 0)
2017-10-28 20:03:38,672 : INFO : min_count=1 leaves 6014253 word corpus (100% of original 6014253, drops 0)
2017-10-28 20:03:38,672 : INFO : deleting the raw counts dictionary of 133 items
2017-10-28 20:03:38,673 : INFO : sample=0.001 downsamples 50 most-common words
2017-10-28 20:03:38,674 : INFO : downsampling leaves estimated 1695942 word corpus (28.2% of prior 6014253)
2017-10-28 20:03:38,6

### keyword extraction

In [38]:
import math
import nltk
from textblob import TextBlob as tb

nltk.download('punkt')

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

[nltk_data] Downloading package punkt to /home/nems/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
bloblist = []
for i in range(len(example_doc)):
    blob = tb(example_doc[i])
    bloblist.append(blob)
print (len(bloblist))

8915


In [41]:
kw_n = 10

key_list = []
for i, blob in enumerate(bloblist[:10]):
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    tmp = ""
    for word, score in sorted_words[ :kw_n]:
        tmp += word + ' '
    key_list.append(tmp)

In [108]:
letter_score = []
for letter in key_list:
    if len(letter) == 0:
        pass
    else:
        score = model.score(letter.split())
        letter_score.append(score.reshape(1, score.shape[0]))

2017-10-28 20:41:26,634 : INFO : scoring sentences with 3 workers on 133 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-28 20:41:26,637 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-28 20:41:26,637 : INFO : scoring 10 sentences took 0.0s, 3881 sentences/s
2017-10-28 20:41:26,638 : INFO : scoring sentences with 3 workers on 133 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-28 20:41:26,639 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-28 20:41:26,640 : INFO : scoring 10 sentences took 0.0s, 6809 sentences/s
2017-10-28 20:41:26,640 : INFO : scoring sentences with 3 workers on 133 vocabulary and 200 features, using sg=0 hs=1 sample=0.001 and negative=0
2017-10-28 20:41:26,642 : INFO : reached end of input; waiting to finish 1 outstanding jobs
2017-10-28 20:41:26,643 : INFO : scoring 10 sentences took 0.0s, 4842 sentences/s
2017-10-28 20:41:26,643 : INFO : scoring

In [109]:
# Cosine Similarity

from scipy import spatial

result = 1 - spatial.distance.cosine(letter_score[0], letter_score[1])
print (result)

0.778173508768


In [110]:
letter_score

[array([[-17.50776291, -19.54599953, -14.43978691, -23.67828941,
          -6.4330616 , -16.28676033, -21.12831688, -40.6035881 ,
         -17.72745132, -39.48973465]], dtype=float32),
 array([[-20.16458511, -26.20037651, -22.28595161, -21.14113045,
         -15.49653721, -18.25351715, -15.35197639,  -7.70956469,
         -24.88040924,  -9.64005661]], dtype=float32),
 array([[-27.44132614, -23.98354149, -26.61302757, -18.2983284 ,
         -18.84646606, -21.57950974, -27.79431725, -13.44624901,
         -16.00395012, -20.61208725]], dtype=float32),
 array([[-23.45058632, -14.10188961, -18.60039711, -21.87195396,
         -19.3116436 , -27.94157028, -21.9922924 ,  -8.9779377 ,
         -19.46103859, -15.48441792]], dtype=float32),
 array([[-11.28796673, -14.58249092, -15.78649426, -17.14369965,
          -3.86019588,  -7.18644762, -12.38673973, -10.57828903]], dtype=float32),
 array([[-53.72369003, -21.31909943, -12.45995522, -30.64398003,
         -18.61462784, -32.48761368, -17.972702